# Дипломчик
### Коды Гоппы

In [1]:
from sage.all import *
import random

In [37]:
class utils:
    def random_binary_unimodular_matrix(rows, cols):
        return random_matrix(GF(2), rows, cols, algorithm='unimodular')
    
    def random_permutation_matrix(n):
        return matrix(GF(2), Permutations(n).random_element().to_matrix())


    def random_irreducible_polynomial(R, degree):
        var = R.gen()

        c = 0
        while True: # todo: benchmark this! how fast we get irreducible poly
            res = 1*var^degree + R.random_element(degree=degree-1)
            c = c + 1

            if res.is_irreducible() == True:
                return res

        # print(f'iterations: {c}') 
        
        return None

    def random_binary_vector(length):
        return vector(random_matrix(GF(2), 1, length))

    def random_binary_err_vector(length, weight_bound):
        list = [0]*length

        weight = 0
        for i in range(length):
            if weight == weight_bound:
                break
        
            rand_elem = random.randint(0,1)
            
            list[i] = rand_elem
            weight = weight + rand_elem

        return vector(GF(2), list)

    def solve_linear_eq(A, b):
        #aug = A.augment(b, subdivide=True)
        #return aug.rref()
        return A.solve_left(b)

    def get_syndrome(R, L, g, y): # todo: we can use H * y^t????
        if len(L) != len(y):
            raise Exception("len(L) != len(y)")

        var = R.gen()
        syndrome = R.zero()
        for i in range(len(y)):
            frac = 0
            if y[i] != 0:
                frac = (var - L[i]) / y[i]
            
            syndrome += utils.get_inverse_by_modulus(frac, g)

        return syndrome.mod(g)
        

    def split(p):
		# split polynomial p over F into even part po
		# and odd part p1 such that p(z) = p2 (z) + z p2 (z)
        Phi = p.parent()
        p0 = Phi([sqrt(c) for c in p.list()[0::2]]);
        p1 = Phi([sqrt(c) for c in p.list()[1::2]]);
        return (p0,p1)
    

    def patterson_algo(R, s, g, q, m):
        h = utils.get_inverse_by_modulus(s, g)
        print(f"h: {h}\n") # TODO: if h == x => sigma = x => stop

        if h == x:
            return vector(GF(2), [0] * q)
        
        # take the necessary square root
        (g0,g1) = utils.split(g)
        sqrt_X = g0*utils.get_inverse_by_modulus(g1, g) # todo: understand!! https://eprint.iacr.org/2007/103.pdf
		        		
        (T0,T1) = utils.split(h + x)
        d = (T0 + sqrt_X*T1).mod(g)

        print(f"h + x mod g: {(h + x).mod(g)}\n")

        # print(f"(h + x)^(2^(mt - 1)) mod g: {((h + x)^(2^(m*g.degree() - 1))).mod(g)}\n")
        print(f"(h + x)^(2^(mt - 1)) mod g: {(((h + x).mod(g))^(2^(m*g.degree() - 1))).mod(g)}\n")
        
        print(f"d: {d}\n")

        print(f"d^2: {d^2}\n")
        print(f"d^2 == h + x mod g: {d^2 == (h + x).mod(g)}\n")

        (a, b) = utils.partial_xgcd(R(d.list()), R(g.list()), R, g.degree())
        print(f"a: {a}\n")
        print(f"b: {b}\n")

        print(f"a == bv mod g: {R(a) == (R(b) * R(d.list())).mod(g)}\n")

        sigma = a * a + x * b * b
        print(f"sigma: {sigma}\n")
        
        print(f"sigma roots: {sigma.roots(R)}\n")
        err_locations = utils.get_error_locations(sigma, L)
        print(f"error locations: {err_locations}\n")

        err_invert = [0] * q
        for err_pos in err_locations:
            err_invert[err_pos] = 1

        return vector(GF(2), err_invert)
        
    

    def get_error_locations(sigma, L):
        E = []
        
        for i in range(len(L)):
            if sigma(L[i]) == 0:
                E.append(i)

        return E


    def get_inverse_by_modulus(poly, mod): # this func works only if poly and mod has no common divisors other than 1
        (g, s, t) = xgcd(poly, mod)
        #if g != 1:
        #    raise Exception("gcd is not 1!")
            
        return s


    def partial_xgcd(a, b, R, bound): # todo: understand stop crit!!!
        s = R.one()
        prev_s = R.zero()

        r = a
        prev_r = b
        
        while not (prev_r.degree() > floor(bound) / 2 and r.degree() == floor(bound/2)):
        # while true:
            q = prev_r.quo_rem(r)[0]
            
            (prev_r, r) = (r, prev_r - q * r)
            (prev_s, s) = (s, prev_s - q * s)
            # (prev_t, t) = (t, prev_t - q * t)

            print(f"r: {r}\n")
            print(f"s: {s}\n")
            # print(f"t: {t}\n")

            # print(f"r == as mod b: {R(r) == R((a * s)).mod(b)}\n")

            print(f"crit: {r.degree()} <= {floor(bound/2)} and {s.degree()} <= {floor((bound - 1)/2)}")
        
            # if not (r.degree() <= floor(bound/2) and s.degree() <= floor((bound - 1)/2)):
            #    return (prev_r, prev_s, prev_t)

        return (r, s)

In [38]:
k = 4
q = 2^k

F = GF(q, 'a')

print(f"elems of field F:\n")
for i,x in enumerate(F):  print("{} {}".format(f'{i+1}:', x))

R.<x> = F[]
RR.<a> = F


n = q
t = 2  # todo: understand how t related to p^m, ans: 2 <= t <= [(2^m - 1) / m] (but why??)

g = utils.random_irreducible_polynomial(R, t)
# g = x^2 + x + a^3


print(f"g: {g}\n")

L = [a for a in random.sample(F.list(), n) if g(a) != 0]
# L = [0, 1]
# for i in range(1,n-1):
#     L.append(a^i)

print(f"elems of L:\n")
for i,l in enumerate(L):  print("{} {}".format(f'{i+1}:', l))

C = codes.GoppaCode(g, L) # todo: sometimes goppa code is defined

# n, k, t - publicly available parameter
k = C.dimension()

print(f'\nn: {n}\nk: {k}\nt: {t}\n')

print(f'{C}\n')

# for i,v in enumerate(C.list()):  print(f"{i+1}: {v}")
# print()


# n - размерность векторного пространства над которым строится код, mt + 1 <= n <= 2^m
# the choice n = 2^m is typical when constructing a Goppa code for a McEliece cryptosystem, but
# Bernstein points out that varying the value of n can yield a better security/eﬃciency trade-oﬀ

# k - размерность пространства кода, для кодов Гоппы справедливо: k >= n - mt
# d - минимальное расстояние кода, для кодов Гоппы справедливо: d >= t + 1, для неприводимого бинарного кода Гоппы: d >= 2t + 1

H = C.parity_check_matrix()
G = C.generator_matrix()

print(f'check matrix:\n{H}\n')
print(f'gen matrix: \n{G}\n')

r = len(G.rows())
c = len(G.columns())

S = utils.random_binary_unimodular_matrix(r, r)

print(f'random binary matrix:\n{S}\n')

P = utils.random_permutation_matrix(c)

print(f'random permutation matrix:\n{P}\n')

GG = S * G * P
print(f"G':\n{GG}\n")

# public key: (G', t)
# private key: (g(x), G, S, P)

# bob sends to alice plaintext m:

m = utils.random_binary_vector(k)
# m = vector(GF(2), [0,1,1,1,0,0,1,1])
print(f'message: {m}\n')

e = utils.random_binary_err_vector(n, t) # todo: wt(e) <= t!!
# e = vector(GF(2), [0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0])
print(f'err vec: {e}\n')

c = (m * GG) + e
print(f'cipher text: {c}\n')
print(f'cipher text without errors: {m * GG}\n')

cc = c * P.inverse()
print(f'mSG + eP^(-1): {cc}\n')
print(f'mSG: {(m * GG) * P.inverse()}\n')

print(f'syndrome(mSG + eP^(-1)): {utils.get_syndrome(R, L, g, cc)}\n')
print(f"e': {e * P.inverse()}\n")

# alice decrypts received ciphertext c:

# c * P^(-1) = m * S * G + e * P^(-1) = m * S * G + e'
# then use Patterson algo to find e' = e * P^(-1)
# then find c' - e' = m * S * G
# L: m' = mS => solve m'G = c' - e' and thus find m'
# then find m = m'S^(-1)

syndrome = utils.get_syndrome(R, L, g, cc)
print(f"syndrome: {syndrome}")

found_errs = utils.patterson_algo(R, syndrome, g, q, k)
print(f"found errors: {found_errs}\n")

cc = cc + found_errs
print(f"corrected cipher text: {cc}\n")

mSS = G.solve_left(cc)
print(f"mS':\n{mSS}\n")

decrypted = mSS * S.inverse()
print(f"decrypted message:\n{decrypted}\n")

print(f"are equal: {decrypted == m}")



elems of field F:

1: 0
2: a
3: a^2
4: a^3
5: a + 1
6: a^2 + a
7: a^3 + a^2
8: a^3 + a + 1
9: a^2 + 1
10: a^3 + a
11: a^2 + a + 1
12: a^3 + a^2 + a
13: a^3 + a^2 + a + 1
14: a^3 + a^2 + 1
15: a^3 + 1
16: 1
g: x^2 + (a^3 + a^2 + a)*x + a^3

elems of L:

1: a^2 + a + 1
2: a^3 + a + 1
3: a^3
4: 1
5: a^2 + a
6: 0
7: a^3 + a^2 + a + 1
8: a + 1
9: a^3 + a
10: a
11: a^3 + 1
12: a^3 + a^2 + 1
13: a^3 + a^2 + a
14: a^3 + a^2
15: a^2
16: a^2 + 1

n: 16
k: 8
t: 2

[16, 8] Goppa code over GF(2)

check matrix:
[1 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0]
[0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1]
[0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 0]
[1 0 0 0 0 1 0 1 0 1 1 1 1 1 0 0]
[0 1 0 0 1 0 0 1 1 1 1 1 1 0 1 0]
[1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 1]
[0 1 1 1 0 0 1 1 0 1 1 0 1 1 1 0]
[1 0 0 0 1 0 0 1 0 1 1 1 0 0 1 1]

gen matrix: 
[1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1]
[0 1 0 0 0 1 0 0 1 0 1 0 1 1 0 1]
[0 0 1 0 0 1 0 0 1 0 0 1 1 1 1 0]
[0 0 0 1 0 1 0 0 0 0 1 1 1 0 1 1]
[0 0 0 0 1 1 0 0 1 0 1 0 0 0 1 1]
[0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0]
[0 0 0 

In [51]:
F = GF(2^4, 'a')

R.<x> = F[]
RR.<a> = F

p1 = x^4 + x + a
p2 = (x + a^3)

print(f"GCD: {gcd(p2, p1)}\n")
print(utils.get_inverse_by_modulus(p2, p1))

# S = R.quotient(p1, 'x')
# print(S(p2))


print(p1.quo_rem(p2))

# print(R.random_element(degree=1).variables()[0]) # get var of poly ring
# utils.random_irreducible_polynomial(R, 4)


print(p1.mod(p2))

print(p1.quo_rem(p2)[1])

p1.sqrt?

# (d,u,v) = xgcd(p2,p1);
# print(u.mod(p2))


GCD: 1

(a^3 + a + 1)*x^3 + (a^2 + a + 1)*x^2 + (a^3 + a^2 + 1)*x + a^3 + 1
(x^3 + a^3*x^2 + (a^3 + a^2)*x + a^3 + a + 1, a^2 + 1)
a^2 + 1
a^2 + 1


Docstring:     
   Compute the square root.

   INPUT:

   * "extend" -- boolean (default: "True"); whether to make a ring
        extension containing a square root if "self" is not a square

   * "all" -- boolean (default: "False"); whether to return a list of
        all square roots or just a square root

   * "name" -- required when "extend=True" and "self" is not a
        square. This will be the name of the generator of the
        extension.

   OUTPUT:

   * if "all=False", a square root; raises an error if "extend=False"
     and "self" is not a square

   * if "all=True", a list of all the square roots (empty if
     "extend=False" and "self" is not a square)

   ALGORITHM:

   It uses "is_square(root=true)" for the hard part of the work, the
   rest is just wrapper code.

   EXAMPLES:

      sage: R.<x> = ZZ[]
      sage: (x^2).sqrt()
      x
      sage: f = x^2 - 4*x + 4; f.sqrt(all=True)
      [x - 2, -x + 2]
      sage: sqrtx = x.sqrt(name="y"); sqrtx
      y
      sage